In [ ]:
# All baseline models are run for inference using this code
# Runs on both CPU/GPU, faster inference on GPU

# Please upload CSFCube-master.zip to the colab notebook
# This zip file is present in our code base

# **Install Libraries and unzip CSF-Cube Repo File**

In [ ]:
!pip install transformers
!pip install -U sentence-transformers
!pip install simcse
from sentence_transformers import SentenceTransformer
from simcse import SimCSE

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=97d42624772379dd0d5b29fc09a471bc340f28a05900dd4ffbe53693198b73ca
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers
Looking in indexes: https://pypi.o

In [ ]:
!unzip CSFCube-master.zip

Archive:  CSFCube-master.zip
7ffe012c1d92c470f185fc086086caf5b8e05ef3
   creating: CSFCube-master/
 extracting: CSFCube-master/.gitignore  
  inflating: CSFCube-master/LICENSE.md  
  inflating: CSFCube-master/README.md  
  inflating: CSFCube-master/abstracts-csfcube-preds.jsonl  
  inflating: CSFCube-master/ann_guidelines.pdf  
  inflating: CSFCube-master/datasheet.md  
   creating: CSFCube-master/eval_scripts/
  inflating: CSFCube-master/eval_scripts/rank_metrics.py  
  inflating: CSFCube-master/eval_scripts/ranking_eval.py  
 extracting: CSFCube-master/eval_scripts/requirements.txt  
   creating: CSFCube-master/eval_scripts/sample_ranked_pools/
  inflating: CSFCube-master/eval_scripts/sample_ranked_pools/test-pid2pool-csfcube-specter-background-ranked.json  
  inflating: CSFCube-master/eval_scripts/sample_ranked_pools/test-pid2pool-csfcube-specter-method-ranked.json  
  inflating: CSFCube-master/eval_scripts/sample_ranked_pools/test-pid2pool-csfcube-specter-result-ranked.json  
  inf

# **Parse CSF-Cube dataset into Python dictionaries**

In [ ]:
import json
def parse_dataset(file_data,file_background,file_method,file_result):
  data = dict()
  with open(file_data,'r') as f:
    abs_list = list(f)
  for paper in abs_list:
    paper = json.loads(paper)
    dict_paper = dict()
    dict_paper['title'] = paper['title']
    dict_paper['abstract'] = paper['abstract']
    dict_paper['pred_labels'] = paper['pred_labels']
    try:
      print(data[paper['paper_id']])
    except:
      data[paper['paper_id']] = dict_paper
  file_list = [file_background,file_method,file_result]
  result_list = []
  for f in file_list:
    with open(f,'r') as json_file:
      results_data = json.load(json_file)
    result_dict = dict()
    for k in results_data.keys():
      try:
        print(result_dict[k].keys())
      except:
        result_dict[k] = dict()
      result_dict[k]['cands'] = results_data[k]['cands']
      result_dict[k]['relevance_adju'] = results_data[k]['relevance_adju']
    result_list.append(result_dict)
  return data,result_list[0],result_list[1],result_list[2]

In [ ]:
sentences,bg_ranking,method_ranking,result_ranking = parse_dataset('/content/CSFCube-master/abstracts-csfcube-preds.jsonl','/content/CSFCube-master/test-pid2anns-csfcube-background.json','/content/CSFCube-master/test-pid2anns-csfcube-method.json','/content/CSFCube-master/test-pid2anns-csfcube-result.json')

# **Load pre-trained language models**

In [ ]:
from transformers import *
import torch
from numpy import dot
from numpy.linalg import norm

def init_model_tokenizer(i):
  if(i==0):
    tokenizer = AutoTokenizer.from_pretrained('allenai/specter')
    model = AutoModel.from_pretrained('allenai/specter')
  if i==3:
    tokenizer = AutoTokenizer.from_pretrained('malteos/scincl')
    model = AutoModel.from_pretrained('malteos/scincl') 
  if i==4:
    tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
    model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')

  if i==10:
    model = SentenceTransformer('paraphrase-TinyBERT-L6-v2')
    tokenizer = None
  if i==11:
    model = SentenceTransformer('nli-roberta-base-v2')
    tokenizer = None
  if i==12:
    model = SimCSE("princeton-nlp/unsup-simcse-bert-base-uncased")
    tokenizer = None
  if i==13:
    model = SimCSE("princeton-nlp/sup-simcse-bert-base-uncased")
    tokenizer = None

  return model,tokenizer

def baseline_model(model_id, model, tokenizer, texts, max_length):
    if model_id == 10 or model_id == 11 or model_id == 12 or model_id == 13 or model_id == 14:
        if model_id == 10 or model_id == 11 or model_id == 14:
          model.eval()
        cls_embeddings = model.encode(texts)
        return cls_embeddings
      
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    with torch.no_grad():
        tokens = tokenizer(texts, max_length=max_length, truncation=True, padding=True, return_tensors='pt')
        tokens.to(device)
        outputs = model(**tokens)

        # Get the output embeddings
        embeddings = outputs.last_hidden_state
        cls_embedding = embeddings[:, 0, :]
        cls_embedding = cls_embedding.cpu().detach().numpy()
        return cls_embedding

/usr/local/lib/python3.9/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(


In [ ]:
keys = [0,3,4,10,11,12,13]

model_dict = dict()
tokenizer_dict = dict()

for k in keys:
  model_dict[k],tokenizer_dict[k] = init_model_tokenizer(k)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allenai--specter/snapshots/5b0bc1dc47329dfaaa100a5c14cf8942b258591e/config.json
Model config BertConfig {
  "_name_or_path": "allenai/specter",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31116
}



loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--allenai--specter/snapshots/5b0bc1dc47329dfaaa100a5c14cf8942b258591e/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--allenai--specter/snapshots/5b0bc1dc47329dfaaa100a5c14cf8942b258591e/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--allenai--specter/snapshots/5b0bc1dc47329dfaaa100a5c14cf8942b258591e/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allenai--specter/snapshots/5b0bc1dc47329dfaaa100a5c14cf8942b258591e/config.json
Model config BertConfig {
  "_name_or_path": "allenai/specter",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act":

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--allenai--specter/snapshots/5b0bc1dc47329dfaaa100a5c14cf8942b258591e/pytorch_model.bin
All model checkpoint weights were used when initializing BertModel.

All the weights of BertModel were initialized from the model checkpoint at allenai/specter.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--malteos--scincl/snapshots/5cab047e2007fcc749ca78c2728142c4df5df1c4/config.json
Model config BertConfig {
  "_name_or_path": "malteos/scincl",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}



loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--malteos--scincl/snapshots/5cab047e2007fcc749ca78c2728142c4df5df1c4/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--malteos--scincl/snapshots/5cab047e2007fcc749ca78c2728142c4df5df1c4/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--malteos--scincl/snapshots/5cab047e2007fcc749ca78c2728142c4df5df1c4/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--malteos--scincl/snapshots/5cab047e2007fcc749ca78c2728142c4df5df1c4/config.json
Model config BertConfig {
  "_name_or_path": "malteos/scincl",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gel

...

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--princeton-nlp--unsup-simcse-bert-base-uncased/snapshots/218baf6bc48c463a53f5b4e1c568299537fab124/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--princeton-nlp--unsup-simcse-bert-base-uncased/snapshots/218baf6bc48c463a53f5b4e1c568299537fab124/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--princeton-nlp--unsup-simcse-bert-base-uncased/snapshots/218baf6bc48c463a53f5b4e1c568299537fab124/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--princeton-nlp--unsup-simcse-bert-base-uncased/snapshots/218baf6bc48c463a53f5b4e1c568299537fab124/config.json
Model config BertConfig {
  "_name_or_path": "princeton-nlp/unsup-simcse-bert-base-uncased",
  "architectures

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--princeton-nlp--unsup-simcse-bert-base-uncased/snapshots/218baf6bc48c463a53f5b4e1c568299537fab124/pytorch_model.bin
All model checkpoint weights were used when initializing BertModel.

All the weights of BertModel were initialized from the model checkpoint at princeton-nlp/unsup-simcse-bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--princeton-nlp--sup-simcse-bert-base-uncased/snapshots/2d82fab19ac3a73a20dd20333d27eb8a52d6e97f/config.json
Model config BertConfig {
  "_name_or_path": "princeton-nlp/sup-simcse-bert-base-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--princeton-nlp--sup-simcse-bert-base-uncased/snapshots/2d82fab19ac3a73a20dd20333d27eb8a52d6e97f/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--princeton-nlp--sup-simcse-bert-base-uncased/snapshots/2d82fab19ac3a73a20dd20333d27eb8a52d6e97f/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--princeton-nlp--sup-simcse-bert-base-uncased/snapshots/2d82fab19ac3a73a20dd20333d27eb8a52d6e97f/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--princeton-nlp--sup-simcse-bert-base-uncased/snapshots/2d82fab19ac3a73a20dd20333d27eb8a52d6e97f/config.json
Model config BertConfig {
  "_name_or_path": "princeton-nlp/sup-simcse-bert-base-uncased",
  "architectures": [
    "

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--princeton-nlp--sup-simcse-bert-base-uncased/snapshots/2d82fab19ac3a73a20dd20333d27eb8a52d6e97f/pytorch_model.bin
All model checkpoint weights were used when initializing BertModel.

All the weights of BertModel were initialized from the model checkpoint at princeton-nlp/sup-simcse-bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.


# **Ranked retrieval for candidates**

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def L2_dist_ranking(cls_embeddings,query_id,cand_ids,results_dict):
  query_cls = cls_embeddings[0]
  if query_id in results_dict.keys():
    print('err')
    return
  results = []
  for i in range(1,len(cls_embeddings)):
    l2_norm = np.linalg.norm(query_cls - cls_embeddings[i])
    results.append([cand_ids[i-1],l2_norm])
  results = sorted(results, key = lambda x: x[1])
  
  for i in range(len(results)):
    results[i][1] = float(results[i][1])
  results_dict[query_id] = results
  return

def cosine_similarity(query_embeddings, cls_embeddings):
  max_cosine_sim = 0.0
  for embd_query in query_embeddings:
    for embd_cand in cls_embeddings:
      curr_cosine_sim = dot(embd_query, embd_cand)/(norm(embd_cand)*norm(embd_query))
      if curr_cosine_sim > max_cosine_sim:
        max_cosine_sim = curr_cosine_sim

  return max_cosine_sim

  

In [ ]:
import json

def write_results(results_dict,model_name,facet):
  file_name = "test-pid2pool-csfcube-" + model_name + "-" + facet + "-ranked.json"
  with open(file_name,'w') as f:
    json.dump(results_dict,f)


In [ ]:
import json
import os

model_map = {'sentbert-pp':10,'sentbert-nli':11, 'unsimcse':12, 'susimcse':13, 'finetuned-sentbert-pp': 14}

def prepare_batch(query_id,data_dict,ranking_dict,query_facet,cand_facet):
  titles = []
  abstracts = []
  query_title = data_dict[query_id]['title']
  titles.append(query_title)
  query_abstract = []
  for i in range(len(data_dict[query_id]['abstract'])):
    curr_label = data_dict[query_id]['pred_labels'][i]
    if query_facet == "abstract" or query_facet == curr_label:
      query_abstract.append(data_dict[query_id]['abstract'][i])
    elif query_facet == "background_label" and curr_label=="objective_label":
      query_abstract.append(data_dict[query_id]['abstract'][i])
    elif query_facet == "objective_label" and curr_label=="background_label":
       query_abstract.append(data_dict[query_id]['abstract'][i])
  abstracts.append(query_abstract)

  for j in range(len(ranking_dict[query_id]['cands'])):
    cand_id = ranking_dict[query_id]['cands'][j]
    cand_title = data_dict[cand_id]['title']
    titles.append(cand_title)
    cand_abstract = []
    for i in range(len(data_dict[cand_id]['abstract'])):
      curr_label = data_dict[cand_id]['pred_labels'][i]
      if cand_facet == "abstract" or cand_facet == curr_label:
        cand_abstract.append(data_dict[cand_id]['abstract'][i])
      elif cand_facet == "background_label" and curr_label=="objective_label":
        cand_abstract.append(data_dict[cand_id]['abstract'][i])
      elif cand_facet == "objective_label" and curr_label=="background_label":
        cand_abstract.append(data_dict[cand_id]['abstract'][i])
    abstracts.append(cand_abstract)    

  return titles,abstracts

def eval_model(model_id,model_name,data_dict,ranking_dict,query_facet,cand_facet,ranking_facet):
  model,tokenizer = model_dict[model_id],tokenizer_dict[model_id]
  results_dict = dict()
  for query_id in ranking_dict.keys():
    batch_data = prepare_batch(query_id,data_dict,ranking_dict,query_facet,cand_facet)
    if model_name == 'specter' or model_name == 'scincl' or model_name == 'scibert_uncased':      
      for i in range(len(batch_data[1])):
        abs = ""
        for j in batch_data[1][i]:
          abs += j
        batch_data[1][i] = abs
      if model_name == 'scibert_uncased':
        title_abs = batch_data[1]
      else:
        title_abs = [batch_data[0][i] + tokenizer.sep_token + batch_data[1][i] for i in range(len(batch_data[0]))]
      cls_embeddings = baseline_model(model_id,model,tokenizer,title_abs,512)
      L2_dist_ranking(cls_embeddings,query_id,ranking_dict[query_id]['cands'],results_dict)
    elif model_name == 'sentbert-pp' or model_name=='sentbert-nli' or model_name == 'unsimcse' or model_name == 'susimcse':
      abstracts = batch_data[1]
      results = []
      if(len(batch_data[1][0])==0):
        batch_data[1][0] = ['']
      query_embeddings = baseline_model(model_map[model_name], model, None, batch_data[1][0], None)
      for i in range(1, len(batch_data[1])):
        if len(batch_data[1][i])==0:
          batch_data[1][i] = ['']
        cls_embeddings = baseline_model(model_map[model_name], model, None, batch_data[1][i], None)
        max_cosine_sim = cosine_similarity(query_embeddings, cls_embeddings)
        results.append([ranking_dict[query_id]['cands'][i-1], max_cosine_sim])
      results = sorted(results, key = lambda x: x[1], reverse=True)
      for i in range(len(results)):
        results[i][1] = float(results[i][1])
      results_dict[query_id] = results

  write_results(results_dict,model_name,ranking_facet)

# **Abstract-level Baselines**

## specter

Query Abstract - Candidate Abstract

In [ ]:
eval_model(0,'specter',sentences,bg_ranking,'abstract','abstract','background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.3086, 0.3063, 0.4755, 0.8338, 0.6582, 0.6546

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2648, 0.3219, 0.5123, 0.8483, 0.6857, 0.6937

/content


In [ ]:
eval_model(0,'specter',sentences,method_ranking,'abstract','abstract','method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1012, 0.1313, 0.3787, 0.6269, 0.3715, 0.3570

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0990, 0.1490, 0.4380, 0.6597, 0.4284, 0.4182

/content


In [ ]:
eval_model(0,'specter',sentences,result_ranking,'abstract','abstract','result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1760, 0.2625, 0.6408, 0.7834, 0.6227, 0.6189

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1770, 0.2590, 0.6028, 0.7520, 0.5716, 0.5886

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1953, 0.2333, 0.4983, 0.7481, 0.5508, 0.5435

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1792, 0.2417, 0.5175, 0.7513, 0.5592, 0.5641

/cont

### Query facet - Candidate Abstract

In [ ]:
eval_model(0,'specter',sentences,bg_ranking,'background_label','abstract','background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2715, 0.3187, 0.4948, 0.8150, 0.6444, 0.6367

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2239, 0.2969, 0.4742, 0.8122, 0.6320, 0.6382

/content


In [ ]:
eval_model(0,'specter',sentences,method_ranking,'method_label','abstract','method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0988, 0.1187, 0.3936, 0.6142, 0.3642, 0.3608

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0992, 0.1260, 0.4094, 0.6393, 0.3982, 0.3995

/content


In [ ]:
eval_model(0,'specter',sentences,result_ranking,'result_label','abstract','result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1681, 0.2562, 0.6215, 0.7620, 0.5802, 0.5738

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1860, 0.2559, 0.6078, 0.7586, 0.5698, 0.5834

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1795, 0.2312, 0.5033, 0.7304, 0.5296, 0.5238

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1694, 0.2252, 0.4978, 0.7354, 0.5318, 0.5388

/cont

### Query facet - Candidate facet

In [ ]:
eval_model(0,'specter',sentences,bg_ranking,'background_label','background_label','background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2971, 0.3500, 0.5271, 0.8234, 0.6725, 0.6674

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2314, 0.3094, 0.4678, 0.8134, 0.6351, 0.6408

/content


In [ ]:
eval_model(0,'specter',sentences,method_ranking,'method_label','method_label','method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1039, 0.1187, 0.4211, 0.6120, 0.3610, 0.3497

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0984, 0.1205, 0.4067, 0.6271, 0.3789, 0.3777

/content


In [ ]:
eval_model(0,'specter',sentences,result_ranking,'result_label','result_label','result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1661, 0.2437, 0.5852, 0.7683, 0.5971, 0.5922

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1730, 0.2524, 0.5883, 0.7563, 0.5736, 0.5845

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1890, 0.2375, 0.5111, 0.7346, 0.5435, 0.5364

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1672, 0.2264, 0.4887, 0.7309, 0.5276, 0.5328

/cont

## scincl

### Query Abstract - Candidate Abstract

In [ ]:
eval_model(3,'scincl',sentences,bg_ranking,'abstract','abstract','background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2748, 0.3625, 0.5409, 0.8250, 0.6739, 0.6708

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2324, 0.3531, 0.5482, 0.8435, 0.6965, 0.7040

/content


In [ ]:
eval_model(3,'scincl',sentences,method_ranking,'abstract','abstract','method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1059, 0.1750, 0.4501, 0.6337, 0.4145, 0.4031

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1086, 0.1736, 0.4747, 0.6809, 0.4708, 0.4661

/content


In [ ]:
eval_model(3,'scincl',sentences,result_ranking,'abstract','abstract','result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2255, 0.2688, 0.6455, 0.8012, 0.6433, 0.6378

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2427, 0.2649, 0.6006, 0.7866, 0.6132, 0.6262

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2021, 0.2687, 0.5455, 0.7533, 0.5772, 0.5705

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1945, 0.2623, 0.5408, 0.7689, 0.5914, 0.5966

/conten

### Query facet - Candidate Abstract

In [ ]:
eval_model(3,'scincl',sentences,bg_ranking,'background_label','abstract','background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.3143, 0.3875, 0.5829, 0.8329, 0.6952, 0.6887

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2481, 0.3469, 0.5228, 0.8284, 0.6752, 0.6798

/content


In [ ]:
eval_model(3,'scincl',sentences,method_ranking,'method_label','abstract','method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0987, 0.1625, 0.3616, 0.6539, 0.4391, 0.4451

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1085, 0.1590, 0.4169, 0.6767, 0.4685, 0.4707

/content


In [ ]:
eval_model(3,'scincl',sentences,result_ranking,'result_label','abstract','result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2171, 0.2875, 0.6605, 0.7921, 0.6325, 0.6170

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2431, 0.2882, 0.6422, 0.7919, 0.6322, 0.6351

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2100, 0.2792, 0.5350, 0.7596, 0.5889, 0.5836

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2001, 0.2636, 0.5284, 0.7646, 0.5908, 0.5940

/conten

### Query facet - Candidate facet

In [ ]:
eval_model(3,'scincl',sentences,bg_ranking,'background_label','background_label','background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.3174, 0.3812, 0.6105, 0.8314, 0.6930, 0.6884

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2551, 0.3562, 0.5628, 0.8327, 0.6822, 0.6869

/content


In [ ]:
eval_model(3,'scincl',sentences,method_ranking,'method_label','method_label','method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0956, 0.1687, 0.4085, 0.6782, 0.4741, 0.4587

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1008, 0.1566, 0.4324, 0.6862, 0.4779, 0.4801

/content


In [ ]:
eval_model(3,'scincl',sentences,result_ranking,'result_label','result_label','result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2389, 0.2812, 0.6721, 0.7704, 0.6209, 0.6290

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2224, 0.2795, 0.6446, 0.7829, 0.6262, 0.6383

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2173, 0.2771, 0.5637, 0.7600, 0.5960, 0.5920

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1923, 0.2626, 0.5469, 0.7662, 0.5940, 0.6003

/conten

## scibert_uncased

### Query Abstract - Candidate Abstract

In [ ]:
eval_model(4,'scibert_uncased',sentences,bg_ranking,'abstract','abstract','background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scibert_uncased --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1879, 0.1938, 0.3383, 0.6612, 0.3965, 0.3948

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1612, 0.2094, 0.3552, 0.7137, 0.4559, 0.4648

/content


In [ ]:
eval_model(4,'scibert_uncased',sentences,method_ranking,'abstract','abstract','method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scibert_uncased --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1181, 0.1188, 0.3318, 0.6122, 0.3394, 0.3514

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1066, 0.1149, 0.3630, 0.6092, 0.3406, 0.3415

/content


In [ ]:
eval_model(4,'scibert_uncased',sentences,result_ranking,'abstract','abstract','result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scibert_uncased --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1142, 0.1625, 0.3763, 0.6907, 0.4520, 0.4551

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1063, 0.1535, 0.3270, 0.6711, 0.4184, 0.4299

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scibert_uncased --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1401, 0.1583, 0.3488, 0.6547, 0.3960, 0.4004

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1242, 0.1580, 0.3480, 0.6629, 0.403

### Query facet - Candidate Abstract

In [ ]:
eval_model(4,'scibert_uncased',sentences,bg_ranking,'background_label','abstract','background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scibert_uncased --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1738, 0.2062, 0.3127, 0.6617, 0.3914, 0.3876

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1526, 0.2000, 0.3054, 0.6775, 0.4031, 0.3995

/content


In [ ]:
eval_model(4,'scibert_uncased',sentences,method_ranking,'method_label','abstract','method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scibert_uncased --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0769, 0.0562, 0.2188, 0.5176, 0.2046, 0.2093

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0927, 0.0670, 0.2413, 0.5371, 0.2308, 0.2372

/content


In [ ]:
eval_model(4,'scibert_uncased',sentences,result_ranking,'result_label','abstract','result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scibert_uncased --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1028, 0.1313, 0.3080, 0.6157, 0.3564, 0.3643

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0964, 0.1378, 0.3050, 0.6043, 0.3407, 0.3580

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scibert_uncased --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1178, 0.1313, 0.2798, 0.5983, 0.3175, 0.3204

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1135, 0.1339, 0.2837, 0.6048, 0.323

### Query facet - Candidate facet

In [ ]:
eval_model(4,'scibert_uncased',sentences,bg_ranking,'background_label','background_label','background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scibert_uncased --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1673, 0.1812, 0.3149, 0.7146, 0.4515, 0.4527

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1505, 0.1906, 0.3220, 0.7181, 0.4450, 0.4592

/content


In [ ]:
eval_model(4,'scibert_uncased',sentences,method_ranking,'method_label','method_label','method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scibert_uncased --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0875, 0.0813, 0.2827, 0.5499, 0.2582, 0.2659

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0855, 0.0990, 0.2786, 0.5724, 0.2891, 0.2970

/content


In [ ]:
eval_model(4,'scibert_uncased',sentences,result_ranking,'result_label','result_label','result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scibert_uncased --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1051, 0.1750, 0.4006, 0.7275, 0.5098, 0.5052

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1053, 0.1792, 0.3655, 0.6782, 0.4421, 0.4444

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scibert_uncased --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1200, 0.1458, 0.3328, 0.6640, 0.4065, 0.4079

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scibert_uncased-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1133, 0.1556, 0.3217, 0.6548, 0.390

# **Sentence-level Baselines**

## sentbert-pp

### Query facet - Candidate Abstract

In [ ]:
eval_model(10, 'sentbert-pp', sentences, bg_ranking, 'background_label', 'abstract', 'background')

In [ ]:
!python3 /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path /content/CSFCube-master/ --ranked_path ./ --experiment sentbert-pp --facet background

EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-sentbert-pp-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2125, 0.2875, 0.4580, 0.7791, 0.5913, 0.5826

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-sentbert-pp-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2124, 0.2875, 0.4667, 0.7917, 0.6033, 0.6084



In [ ]:
eval_model(10, 'sentbert-pp', sentences, method_ranking, 'method_label', 'abstract', 'method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0891, 0.1000, 0.3676, 0.5629, 0.2963, 0.3016

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1000, 0.1083, 0.3630, 0.5950, 0.3281, 0.3340

/content


In [ ]:
eval_model(10, 'sentbert-pp', sentences, result_ranking, 'result_label', 'abstract', 'result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1423, 0.2187, 0.4953, 0.7686, 0.5843, 0.5833

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1360, 0.1983, 0.4173, 0.7190, 0.5081, 0.5235

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1480, 0.2021, 0.4403, 0.7035, 0.4906, 0.4892

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1483, 0.1962, 0.4141, 0.6999, 0.4770, 0.485

### Query Abstract - Candidate Abstract

In [ ]:
eval_model(10, 'sentbert-pp', sentences, bg_ranking, 'abstract', 'abstract', 'background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2068, 0.3125, 0.5012, 0.7738, 0.5826, 0.5787

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1951, 0.2938, 0.4754, 0.7869, 0.5911, 0.5938

/content


In [ ]:
eval_model(10, 'sentbert-pp', sentences, method_ranking, 'abstract', 'abstract', 'method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0923, 0.1375, 0.3594, 0.5816, 0.3205, 0.3166

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0982, 0.1354, 0.3973, 0.6103, 0.3581, 0.3610

/content


In [ ]:
eval_model(10, 'sentbert-pp', sentences, result_ranking, 'abstract', 'abstract', 'result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1548, 0.2188, 0.4771, 0.7638, 0.5815, 0.5837

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1409, 0.2010, 0.4536, 0.7232, 0.5141, 0.5239

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1513, 0.2229, 0.4459, 0.7064, 0.4949, 0.4930

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1438, 0.2086, 0.4419, 0.7050, 0.4855, 0.490

### Query facet - Candidate facet

In [ ]:
eval_model(10, 'sentbert-pp', sentences, bg_ranking, 'background_label', 'background_label', 'background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2117, 0.2875, 0.4181, 0.7946, 0.6038, 0.6097

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2192, 0.3031, 0.4681, 0.8002, 0.6136, 0.6266

/content


In [ ]:
eval_model(10, 'sentbert-pp', sentences, method_ranking, 'method_label', 'method_label', 'method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0769, 0.1125, 0.2604, 0.5765, 0.3282, 0.3431

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0877, 0.1063, 0.2579, 0.5850, 0.3107, 0.3259

/content


In [ ]:
eval_model(10, 'sentbert-pp', sentences, result_ranking, 'result_label', 'result_label', 'result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1223, 0.2313, 0.5345, 0.7570, 0.5751, 0.5730

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1122, 0.2156, 0.4921, 0.7206, 0.5192, 0.5298

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1370, 0.2104, 0.4043, 0.7094, 0.5024, 0.5086

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1381, 0.2062, 0.4039, 0.6998, 0.4780, 0.490

## sentbert-nli

### Query facet - Candidate Abstract

In [ ]:
eval_model(11, 'sentbert-nli', sentences, bg_ranking, 'background_label', 'abstract', 'background')

In [ ]:
!python3 /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path /content/CSFCube-master/ --ranked_path ./ --experiment sentbert-nli --facet background

EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-sentbert-nli-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1942, 0.2625, 0.4174, 0.7597, 0.5542, 0.5539

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-sentbert-nli-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1902, 0.2500, 0.4013, 0.7580, 0.5345, 0.5423



In [ ]:
eval_model(11, 'sentbert-nli', sentences, method_ranking, 'method_label', 'abstract', 'method')

In [ ]:
!python3 /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path /content/CSFCube-master/ --ranked_path ./ --experiment sentbert-nli --facet method

EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-sentbert-nli-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0837, 0.1125, 0.2262, 0.5666, 0.2864, 0.2917

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-sentbert-nli-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0911, 0.1146, 0.2890, 0.5852, 0.3068, 0.3110



In [ ]:
eval_model(11, 'sentbert-nli', sentences, result_ranking, 'result_label', 'abstract', 'result')

In [ ]:
!python3 /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path /content/CSFCube-master/ --ranked_path ./ --experiment sentbert-nli --facet result

EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-sentbert-nli-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1422, 0.2188, 0.5232, 0.7465, 0.5474, 0.5370

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-sentbert-nli-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1423, 0.2205, 0.4699, 0.7213, 0.5083, 0.5130



In [ ]:
!python3 /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path /content/CSFCube-master/ --ranked_path ./ --experiment sentbert-nli --facet all

EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-sentbert-nli-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1400, 0.1979, 0.3889, 0.6910, 0.4627, 0.4609

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-sentbert-nli-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1404, 0.1942, 0.3867, 0.6868, 0.4484, 0.4539



### Query Abstract - Candidate Abstract

In [ ]:
eval_model(11, 'sentbert-nli', sentences, bg_ranking, 'abstract', 'abstract', 'background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-nli --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-nli-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2174, 0.2750, 0.4503, 0.7681, 0.5576, 0.5576

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-nli-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1934, 0.2406, 0.3872, 0.7559, 0.5235, 0.5360

/content


In [ ]:
eval_model(11, 'sentbert-nli', sentences, method_ranking, 'abstract', 'abstract', 'method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-nli --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-nli-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0930, 0.1125, 0.2842, 0.5937, 0.3137, 0.3244

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-nli-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0896, 0.1007, 0.2595, 0.6004, 0.3170, 0.3241

/content


In [ ]:
eval_model(11, 'sentbert-nli', sentences, result_ranking, 'abstract', 'abstract', 'result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-nli --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-nli-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1352, 0.1812, 0.3850, 0.7007, 0.4681, 0.4710

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-nli-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1362, 0.1990, 0.4765, 0.6929, 0.4626, 0.4665

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-nli --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-nli-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1485, 0.1896, 0.3732, 0.6875, 0.4465, 0.4510

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-nli-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1390, 0.1794, 0.3754, 0.6818, 0.4331, 0.4

### Query facet - Candidate facet

In [ ]:
eval_model(11, 'sentbert-nli', sentences, bg_ranking, 'background_label', 'background_label', 'background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-nli --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-nli-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1901, 0.2750, 0.4052, 0.7654, 0.5517, 0.5522

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-nli-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1839, 0.2719, 0.4086, 0.7677, 0.5422, 0.5523

/content


In [ ]:
eval_model(11, 'sentbert-nli', sentences, method_ranking, 'method_label', 'method_label', 'method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-nli --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-nli-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0786, 0.0938, 0.1927, 0.5449, 0.2527, 0.2664

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-nli-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0851, 0.1108, 0.2428, 0.5622, 0.2781, 0.2868

/content


In [ ]:
eval_model(11, 'sentbert-nli', sentences, result_ranking, 'result_label', 'result_label', 'result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-nli --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-nli-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1222, 0.1938, 0.4509, 0.7438, 0.5266, 0.5324

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-nli-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1176, 0.1997, 0.4127, 0.7164, 0.4957, 0.5051

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-nli --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-nli-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1303, 0.1875, 0.3496, 0.6847, 0.4437, 0.4503

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-nli-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1279, 0.1928, 0.3537, 0.6803, 0.4367, 0.4

## unsimcse


### Query facet - Candidate Abstract

In [ ]:
eval_model(12, 'unsimcse', sentences, bg_ranking, 'background_label', 'abstract', 'background')

100%|██████████| 1/1 [00:00<00:00, 35.89it/s]


In [ ]:
!python3 /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path /content/CSFCube-master/ --ranked_path ./ --experiment unsimcse --facet background

EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-unsimcse-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2166, 0.2000, 0.3188, 0.7343, 0.4969, 0.4984

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-unsimcse-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1936, 0.2313, 0.3662, 0.7547, 0.5154, 0.5269



In [ ]:
eval_model(12, 'unsimcse', sentences,  method_ranking, 'method_label', 'abstract', 'method')

100%|██████████| 1/1 [00:00<00:00, 44.47it/s]


In [ ]:
!python3 /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path /content/CSFCube-master/ --ranked_path ./ --experiment unsimcse --facet method

EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-unsimcse-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0896, 0.1000, 0.3400, 0.5677, 0.3044, 0.2956

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-unsimcse-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0871, 0.0972, 0.2844, 0.5875, 0.3175, 0.3180



In [ ]:
eval_model(12, 'unsimcse', sentences, result_ranking, 'result_label', 'abstract', 'result')

100%|██████████| 1/1 [00:00<00:00, 54.80it/s]


In [ ]:
!python3 /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path /content/CSFCube-master/ --ranked_path ./ --experiment unsimcse --facet result

EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-unsimcse-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1336, 0.2438, 0.5591, 0.7308, 0.5380, 0.5333

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-unsimcse-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1196, 0.2024, 0.4148, 0.6908, 0.4601, 0.4697



In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment unsimcse --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-unsimcse-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1466, 0.1812, 0.4060, 0.6776, 0.4464, 0.4424

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-unsimcse-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1325, 0.1752, 0.3530, 0.6758, 0.4287, 0.4359

/co

### Query Abstract - Candidate Abstract

In [ ]:
eval_model(12, 'unsimcse', sentences, bg_ranking, 'abstract', 'abstract', 'background')

100%|██████████| 1/1 [00:00<00:00, 32.25it/s]


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment unsimcse --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-unsimcse-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2238, 0.2563, 0.4274, 0.7725, 0.5501, 0.5527

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-unsimcse-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1920, 0.2469, 0.3976, 0.7735, 0.5440, 0.5510

/content


In [ ]:
eval_model(12, 'unsimcse', sentences, method_ranking, 'abstract', 'abstract', 'method')

100%|██████████| 1/1 [00:00<00:00, 49.19it/s]


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment unsimcse --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-unsimcse-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0986, 0.0875, 0.2165, 0.5749, 0.2937, 0.2916

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-unsimcse-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0930, 0.0799, 0.2018, 0.5923, 0.3096, 0.3141

/content


In [ ]:
eval_model(12, 'unsimcse', sentences, result_ranking, 'abstract', 'abstract', 'result')

100%|██████████| 1/1 [00:00<00:00, 43.14it/s]


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment unsimcse --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-unsimcse-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1394, 0.2062, 0.4730, 0.7337, 0.5326, 0.5303

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-unsimcse-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1215, 0.1892, 0.4195, 0.6947, 0.4692, 0.4791

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment unsimcse --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-unsimcse-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1539, 0.1833, 0.3723, 0.6937, 0.4588, 0.4582

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-unsimcse-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1347, 0.1705, 0.3385, 0.6850, 0.4387, 0.4459

/co

### Query facet - Candidate facet

In [ ]:
eval_model(12, 'unsimcse', sentences, bg_ranking, 'background_label', 'background_label', 'background')

100%|██████████| 1/1 [00:00<00:00, 39.75it/s]


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment unsimcse --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-unsimcse-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1811, 0.2375, 0.3298, 0.7578, 0.5442, 0.5404

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-unsimcse-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1721, 0.2688, 0.4027, 0.7725, 0.5574, 0.5673

/content


In [ ]:
eval_model(12, 'unsimcse', sentences, method_ranking, 'method_label', 'method_label', 'method')

100%|██████████| 1/1 [00:00<00:00, 49.50it/s]


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment unsimcse --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-unsimcse-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0934, 0.1000, 0.3333, 0.5538, 0.2672, 0.2791

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-unsimcse-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0890, 0.0889, 0.2470, 0.5566, 0.2645, 0.2776

/content


In [ ]:
eval_model(12, 'unsimcse', sentences, result_ranking, 'result_label', 'result_label', 'result')

100%|██████████| 1/1 [00:00<00:00, 47.03it/s]


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment unsimcse --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-unsimcse-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1320, 0.2000, 0.4494, 0.7361, 0.5264, 0.5250

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-unsimcse-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1157, 0.1667, 0.3586, 0.6988, 0.4616, 0.4744

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment unsimcse --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-unsimcse-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1355, 0.1792, 0.3708, 0.6826, 0.4459, 0.4482

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-unsimcse-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1247, 0.1723, 0.3328, 0.6737, 0.4247, 0.4367

/co

## susimcse

### Query facet - Candidate Abstract

In [ ]:
eval_model(13, 'susimcse', sentences, bg_ranking, 'background_label', 'abstract', 'background')

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
!python3 /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path /content/CSFCube-master/ --ranked_path ./ --experiment susimcse --facet background

In [ ]:
eval_model(13, 'susimcse', sentences, method_ranking, 'method_label', 'abstract', 'method')

100%|██████████| 1/1 [00:00<00:00, 55.93it/s]


In [ ]:
!python3 /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path /content/CSFCube-master/ --ranked_path ./ --experiment susimcse --facet method

EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-susimcse-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0787, 0.1063, 0.3884, 0.5815, 0.3039, 0.3049

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-susimcse-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0858, 0.0976, 0.2901, 0.5854, 0.2973, 0.3088



In [ ]:
eval_model(13, 'susimcse', sentences, result_ranking, 'result_label', 'abstract', 'result')

100%|██████████| 1/1 [00:00<00:00, 57.54it/s]


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment susimcse --facet result
%cd ..

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment susimcse --facet all
%cd ..

### Query Abstract - Candidate Abstract

In [ ]:
eval_model(13, 'susimcse', sentences, bg_ranking, 'abstract', 'abstract', 'background')

100%|██████████| 1/1 [00:00<00:00, 27.34it/s]


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment susimcse --facet background
%cd ..

In [ ]:
eval_model(13, 'susimcse', sentences, method_ranking, 'abstract', 'abstract', 'method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment susimcse --facet method
%cd ..

In [ ]:
eval_model(13, 'susimcse', sentences, result_ranking, 'abstract', 'abstract', 'result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment susimcse --facet result
%cd ..

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment susimcse --facet all
%cd ..

### Query facet - Candidate facet

In [ ]:
eval_model(13, 'susimcse', sentences, bg_ranking, 'background_label', 'background_label', 'background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment susimcse --facet background
%cd ..

In [ ]:
eval_model(13, 'susimcse', sentences, method_ranking, 'method_label', 'method_label', 'method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment susimcse --facet method
%cd ..

In [ ]:
eval_model(13, 'susimcse', sentences, result_ranking, 'result_label', 'result_label', 'result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment susimcse --facet result
%cd ..

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment susimcse --facet all
%cd ..